In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, sum as spark_sum, to_date

# Initialize Spark session
spark = SparkSession.builder.appName("AccountsPayableReceivableETL").getOrCreate()

# Load data from Azure Data Lake
ap_df = spark.read.csv("abfss://datalake@myaccount.dfs.core.windows.net/accounts-payable/*.csv", header=True, inferSchema=True)
ar_df = spark.read.csv("abfss://datalake@myaccount.dfs.core.windows.net/accounts-receivable/*.csv", header=True, inferSchema=True)
bank_df = spark.read.csv("abfss://datalake@myaccount.dfs.core.windows.net/bank-transactions/*.csv", header=True, inferSchema=True)

# Standardize date format
ap_df = ap_df.withColumn("TransactionDate", to_date(col("TransactionDate"), "yyyy-MM-dd"))
ar_df = ar_df.withColumn("TransactionDate", to_date(col("TransactionDate"), "yyyy-MM-dd"))
bank_df = bank_df.withColumn("TransactionDate", to_date(col("TransactionDate"), "yyyy-MM-dd"))

# Join AP and AR with bank transactions for reconciliation
reconciled_df = ap_df.join(bank_df, (ap_df.Amount == bank_df.Amount) & (ap_df.TransactionDate == bank_df.TransactionDate), "left")     .withColumn("Reconciled", when(col("bank.TransactionID").isNotNull(), "Yes").otherwise("No"))

# Calculate outstanding and overdue transactions
outstanding_df = ar_df.groupBy("CustomerID").agg(spark_sum("Amount").alias("OutstandingReceivable"))
overdue_df = ap_df.filter(col("DueDate") < col("TransactionDate")).groupBy("VendorID").agg(spark_sum("Amount").alias("OverduePayable"))

# Save transformed data to Azure Data Lake
reconciled_df.write.parquet("abfss://datalake@myaccount.dfs.core.windows.net/processed_data/reconciled_transactions/")
outstanding_df.write.parquet("abfss://datalake@myaccount.dfs.core.windows.net/processed_data/outstanding_receivables/")
overdue_df.write.parquet("abfss://datalake@myaccount.dfs.core.windows.net/processed_data/overdue_payables/")
